In [7]:
import cv2
from util import plot_images, read_images
from preprocessing import preprocess_image
import numpy as np
%load_ext autoreload
%autoreload 2

test_malignant_folder_path = "data/test/malignant"

test_malignants = read_images(test_malignant_folder_path)

preprocessed_image_test_malignants = [preprocess_image(image) for image in test_malignants]

print(preprocessed_image_test_malignants.__len__())

# save preprocessed images
for i, image in enumerate(preprocessed_image_test_malignants):
    cv2.imwrite("data/preprocessed/test/malignant/{}.jpg".format(i), image)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
300
